# Agent Memory

Short-term memory (conversation history) and long-term memory (user facts across sessions).

In [ ]:
import sys
sys.path.append('D:/Courses/Udemy/AI Agent Projects')

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.agents import create_agent
from langchain.messages import HumanMessage, SystemMessage
from langchain.tools import tool
from scripts import base_tools

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

## Problem: No Memory

In [ ]:
agent_no_memory = create_agent(model=model)

agent_no_memory.invoke({'messages': [HumanMessage("My name is John")]})
r = agent_no_memory.invoke({'messages': [HumanMessage("What's my name?")]})

r['messages'][-1].text

## Short-Term Memory: SQLite

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

os.makedirs("db", exist_ok=True)
conn = sqlite3.connect("db/checkpoints.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    checkpointer=checkpointer
)

In [ ]:
config = {"configurable": {"thread_id": "user_123"}}

agent.invoke({'messages': [HumanMessage("My name is John")]}, config)
r = agent.invoke({'messages': [HumanMessage("What's my name?")]}, config)

r['messages'][-1].text

In [ ]:
# Multiple sessions
session_a = {"configurable": {"thread_id": "session_a"}}
session_b = {"configurable": {"thread_id": "session_b"}}

agent.invoke({'messages': [HumanMessage("I like Python")]}, session_a)
agent.invoke({'messages': [HumanMessage("I like JavaScript")]}, session_b)

r_a = agent.invoke({'messages': [HumanMessage("What do I like?")]}, session_a)
r_b = agent.invoke({'messages': [HumanMessage("What do I like?")]}, session_b)

print("A:", r_a['messages'][-1].text)
print("B:", r_b['messages'][-1].text)

## Short-Term Memory: PostgreSQL

In [ ]:
from langgraph.checkpoint.postgres import PostgresSaver
import psycopg

with PostgresSaver.from_conn_string(os.getenv("POSTGRESQL_URL")) as pg_checkpointer:
    agent_pg = create_agent(
        model=model,
        tools=[base_tools.web_search, base_tools.get_weather],
        checkpointer=pg_checkpointer
    )
    
    config = {"configurable": {"thread_id": "postgres_session"}}
    
    agent_pg.invoke({'messages': [HumanMessage("My city is Mumbai")]}, config)
    r = agent_pg.invoke({'messages': [HumanMessage("What's my city?")]}, config)
    
    r['messages'][-1].text

## Long-Term Memory: PostgresStore

In [ ]:
from langgraph.store.postgres import PostgresStore

embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

def embed(texts: list[str]):
    return embeddings.embed_documents(texts)

pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True, prepare_threshold=0)
store = PostgresStore(pg_conn, index={"embed": embed, "dims": 768})
store.setup()

In [ ]:
# Memory tools
@tool
def save_user_memory(user_id: str, category: str, information: dict) -> str:
    """Save user information to long-term memory."""
    store.put((user_id, "preferences"), category, information)
    return "saved"

@tool
def get_user_memory(user_id: str, category: str) -> str:
    """Retrieve user information from long-term memory."""
    item = store.get((user_id, "preferences"), category)
    return str(item.value) if item else "not found"

In [ ]:
# Agent with long-term memory tools
pg_saver = PostgresSaver(pg_conn)
pg_saver.setup()

agent_ltm = create_agent(
    model=model,
    tools=[base_tools.web_search, save_user_memory, get_user_memory],
    checkpointer=pg_saver
)

config = {"configurable": {"thread_id": "ltm_session"}}

# Save preferences
agent_ltm.invoke({
    'messages': [HumanMessage("Save: I'm John from NYC, I prefer Python")]
}, config)

In [ ]:
# Retrieve in new session
new_config = {"configurable": {"thread_id": "new_session"}}

r = agent_ltm.invoke({
    'messages': [HumanMessage("What do you know about user john?")]
}, new_config)

r['messages'][-1].text

In [ ]:
# Semantic search
memories = store.search(("john", "preferences"), query="programming", limit=3)

for m in memories:
    print(f"{m.key}: {m.value}")

## Memory Comparison

| Type | Storage | Use Case | Persistence |
|------|---------|----------|-------------|
| **Short-term** | Checkpointer | Conversation history | Session |
| **Long-term** | Store | User preferences, facts | Cross-session |

## Key Takeaways

- Short-term: SQLite (dev), PostgreSQL (prod)
- Long-term: PostgresStore with embeddings
- Thread IDs manage sessions
- Semantic search finds relevant memories
- Memory tools enable save/retrieve operations

In [ ]:
# Cleanup
pg_conn.close()